In [1]:
import pulp
import itertools

## Parameters

In [ ]:
troops = 5
battlefileds = 3

## Strategies generation

In [3]:
def genearte_distributions(troops: int, 
                           battlefileds: int) -> list:
    
    """Function for the possible distribution generation"""
    
    if battlefileds == 1:
        return [[troops]]
    
    distributions = []
    for i in range(troops + 1):
        for rest in genearte_distributions(troops - i, battlefileds - 1):
            distributions.append([i] + rest)
    return distributions

In [4]:
my_strategies = genearte_distributions(troops, battlefileds)
opponent_strategies = my_strategies.copy()

## Setting up teh solver 

In [7]:
# Probabilities for each strategy
prob_vars = [pulp.LpVariable(f"p_{i}", lowBound=0) for i in range(len(my_strategies))]

In [9]:
# Game value
v = pulp.LpVariable("v")

In [11]:
# Problem
prob  = pulp.LpProblem("Battle_1", pulp.LpMaximize)
prob += v

In [12]:
# Constraints
for j, opp_strat in enumerate(opponent_strategies):
    payoff_expr = 0
    for i, my_start in enumerate(my_strategies):
        wins = sum(1 if my_start[b] > opp_strat[b] else 0 for b in range(battlefileds))
        payoff_expr += wins * prob_vars[i]
    prob += (payoff_expr >= v)

In [13]:
prob

Battle_1:
MAXIMIZE
1*v + 0
SUBJECT TO
_C1: p_1 + 2 p_10 + p_11 + 2 p_12 + 2 p_13 + 2 p_14 + p_15 + 2 p_16 + 2 p_17
 + p_18 + 2 p_19 + p_2 + p_20 + p_3 + p_4 + p_5 + p_6 + 2 p_7 + 2 p_8 + 2 p_9
 - v >= 0

_C2: p_0 + 2 p_10 + p_11 + p_12 + 2 p_13 + 2 p_14 + p_15 + p_16 + 2 p_17
 + p_18 + p_19 + p_2 + p_20 + p_3 + p_4 + p_5 + p_6 + p_7 + 2 p_8 + 2 p_9 - v
 >= 0

_C3: p_0 + p_1 + 2 p_10 + p_11 + p_12 + p_13 + 2 p_14 + p_15 + p_16 + p_17
 + p_18 + p_19 + p_20 + p_3 + p_4 + p_5 + 2 p_6 + p_7 + p_8 + 2 p_9 - v >= 0

_C4: p_0 + p_1 + 2 p_10 + 2 p_11 + p_12 + p_13 + p_14 + p_15 + p_16 + p_17
 + p_18 + p_19 + p_2 + p_20 + p_4 + p_5 + 2 p_6 + 2 p_7 + p_8 + p_9 - v >= 0

_C5: p_0 + p_1 + p_10 + 2 p_11 + 2 p_12 + p_13 + p_14 + 2 p_15 + p_16 + p_17
 + p_18 + p_19 + p_2 + p_20 + p_3 + p_5 + 2 p_6 + 2 p_7 + 2 p_8 + p_9 - v >= 0

_C6: p_0 + p_1 + p_10 + 2 p_11 + 2 p_12 + 2 p_13 + p_14 + 2 p_15 + 2 p_16
 + p_17 + 2 p_18 + p_19 + p_2 + p_20 + p_3 + p_4 + 2 p_6 + 2 p_7 + 2 p_8
 + 2 p_9 - v >= 0

_C7: p_0 

In [14]:
# Solve
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /root/miniconda3/envs/py310lin/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/576e04906fe4467ca0094fc97eb18c9d-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/576e04906fe4467ca0094fc97eb18c9d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 26 COLUMNS
At line 469 RHS
At line 491 BOUNDS
At line 493 ENDATA
Problem MODEL has 21 rows, 22 columns and 441 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve thinks problem is unbounded
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 1 - largest nonzero change 0 ( 0%) - largest zero change 1.0142706e-05
0  Obj -0 Dual inf 0.0099999 (1) w.o. free dual inf (0)
0  Obj -0 Dual inf 0.0099999 (1) w.o. free dual inf (0)
4  Obj 7e-12 Dual inf 17.666665 (13)
4  Obj 7e-12 Dual inf 17.666665 (13)
D

-2

## Results

In [17]:
print(f"Status: {pulp.LpStatus[prob.status]}")
for var in prob_vars:
    if var.varValue > 1e-6:
        idx = int(var.name.split('_')[1])
        print(f"Startegy {my_strategies[idx]}: probability {var.varValue:.4f}")
print(f"Game value: {v.varValue}")

Status: Unbounded
Game value: 7e-12
